# **UNICEF Indicator Data Warehouse** - SDMX REST API Web Service

In [0]:
Test

In [0]:
# import necessary libraries 
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint

Connecting to the UNICEF's data warehouse using the **Statistical Data and Metadata eXchange** (SDMX) standard.

In [0]:
endpoint = "https://sdmx.data.unicef.org/ws/public/sdmxapi/rest"

## **1. STEP: select a Data Flow**

Data Flows contain a set of data with a common structure and usually group similar topics. So I first use the API to get a list of all those Data Flows. 

This requires adding the following to the endpoint and providing the parameters for ```format```, ```detail``` and ```reference```. I chose to get the data in the **JSON format**, with **full detail** and **no references**. 

In [0]:
dataflow_endpoint = "/dataflow/all/all/latest"
dataflow_params = {
    "format": "sdmx-json",
    "detail": "full",
    "reference": "none",
}

In [0]:
# get the response and checking the status code 
dataflow_response = requests.get(url=endpoint+dataflow_endpoint, params=dataflow_params)
print(dataflow_response.status_code)

# converting the response into JSON format, printing the response  
dataflows = dataflow_response.json()['data']['dataflows']
# uncomment the next line to see the complete response 
# pprint(dataflows)

In [0]:
# this prints out all the Data Flows and their necessary keys
for dflow in dataflows:
    # those values will be necessary for the next API call
    print(f"name: {dflow['name']} - id: {dflow['id']} - agency-id: {dflow['agencyID']} - dataflow-version: {dflow['version']}")

I chose  ```name: Demography - id: DM - agency-id: UNICEF - dataflow-version: 1.0```  for further exploration.

To get hold of the actual data later, it's necessary to remember the **id**, the **id of the agency** and the **version** of the chosen Data Flow. 

In [0]:
SELECTED_DFLOW = 'DEMOGRAPHY'
SELECTED_DFLOW_ID = 'DM'
SELECTED_AGENCY_ID = 'UNICEF'
SELECTED_DFLOW_VER = "1.0"

## **2. STEP: explore the structure of the selected Data Flow**

This structure uses the **Data Structure Definition** (DSD) to achieve reusability between different Data Flows. It comprises concepts that can have the usage roles of a dimension, an attribute or a measure. 

A new addition to the endpoint, including the agency ID of the selected Data Flow, must be made to get hold of the structure. The parameters that are sent with the API call stay the same.

In [0]:
structure_endpoint = f"/datastructure/{SELECTED_AGENCY_ID}/{SELECTED_DFLOW}/latest"
structure_params = {
    "format": "sdmx-json",
    "detail": "full",
    "reference": "none",
}

In [0]:
# get the response and checking the status code 
structure_response = requests.get(url=endpoint+structure_endpoint, params=structure_params)
print(dataflow_response.status_code)

# get hold of the dataStructures 
selected_structure = structure_response.json()['data']['dataStructures'][0]

# display the structure in JSON format 
pprint(selected_structure)


Let's explore the concepts that were contained in the API response at little bit more. The key to access those lists is the ```dataStructureComponents``` key.

In [0]:
datastructure_comp = selected_structure['dataStructureComponents']

### Dimensions

In [0]:
# get the dimensions
dimensions_json = datastructure_comp['dimensionList']

dimensions = [(dim['id'], dim['position']) for dim in dimensions_json['dimensions']]
print(f'dimensions: {dimensions}')

time_dimensions = [(tdim['id'],tdim['position']) for tdim in dimensions_json['timeDimensions']]
print(f'time dimensions: {time_dimensions}')

The ```dataStructureComponents``` for the Demography-Data Flow includes **6 dimensions total**, with the ```TIME_PERIOD``` in its own list of ```timeDimensions```. The positions at the second position in the tuple are required when making queries for specific data slices later. 

### Attributes

In [0]:
# get the attributes
attributes_json = datastructure_comp['attributeList']
attributes = attributes_json['attributes']

for att in attributes:
    print(f"{att['id']} - relation: {att['attributeRelationship']}")

There are **11 different attributes** that can be attached to different levels in the Demography-Data Flow.


## **3. STEP: query the desired slice of data**

In [0]:
complete_data_endpoint = f"/data/{SELECTED_AGENCY_ID},{SELECTED_DFLOW_ID},{SELECTED_DFLOW_VER}/all"
complete_data_params = {
    "format": "sdmx-json"
}

This would get hold of the complete dataset within this chosen Data Flow and version, but it's also possible to get just a piece of this dataset.

Getting just a slice of the data is possible by specifying specific values for the dimensions. Let's say we want to select the following:

- for the ```REF_AREA``` **Western Europe**
- for the ```INDICATOR``` **everything**
- for the ```RESIDENCE``` **just the total**
- for the ```SEX``` **just the total**
- for the ```AGE``` **everything**

The possible values for those dimensions can be found in the **Codelist** for this Data Flow. To get hold of this Codelist, another API call is necessary. The string that has to be added to the endpoint is the same as for getting the complete data, but with the parameters, the ```detail``` is limited to ```structureOnly```.

In [0]:
codelist_endpoint = f"/data/{SELECTED_AGENCY_ID},{SELECTED_DFLOW_ID},{SELECTED_DFLOW_VER}/all"
codelist_params = {
    "format": "sdmx-json",
    "detail": "structureOnly"
}

In [0]:
# get the response and checking the status code 
codelist_response = requests.get(url=endpoint+codelist_endpoint, params=codelist_params)
print(codelist_response.status_code)

codelist = codelist_response.json()['structure']['dimensions']['observation']
# uncomment next line to print codelist
# pprint(codelist)

The following picture is a screenshot from the Codelist returned from the API call. The Codelist provides a list of all the dimensions and all possible ```values``` for them. Because I wanted to limited the ```REF_AREA```, ```RESIDENCE``` and ```SEX``` let's look at the possible values for those: 


![codelist_detail.png](attachment:codelist_detail.png)

Looking for the ```REF_AREA``` to select only Western Europe. The order of dimensions will be the same as before; therefore, I know that the ```REF_AREA``` will be the first dimension in the list.

In [0]:
ref_area_values = [(v['name'], v['id']) for v in codelist[0]['values']]   
# this prints a list of tuples containing the name of the country or region and the id 
print(ref_area_values)

# search for the id of Western Europe
west_eu_id = [i for (n, i) in ref_area_values if n == "Western Europe"]
print(f"ID of Western Europe: {west_eu_id[0]}")


Next let's get the value for ```RESIDENCE``` which should be third in the list:

In [0]:
residence_values = [(v['name'], v['id']) for v in codelist[2]['values']]
print(residence_values)

And at last the value for ```SEX``` which should be fourth: 

In [0]:
sex_values = [(v['name'], v['id']) for v in codelist[3]['values']]
print(sex_values)

Now those dimension values can be used in the query string:

- select **Western Europe** as reference area: ```UNICEF_WE```
- select **all residencies**: ```_T```
- select **all genders**: ```_T```

Those parameters have to be added to the endpoint string **separated by dots**. Multiple values for a dimension should be separated by a "+". 

**Important**: The query string has to contain every dimension. If nothing should be specified, it has to be treated like being left out by just typing the next separating dot. The order is the same as in the dimensions list shown above or as specified under their ```position``` attribute. 

**REF_AREA.~~INDICATOR~~.RESIDENCE.SEX.~~AGE~~** -> final string: **UNICEF_WE.._T._T.**

In [0]:
QUERY_STR = "UNICEF_WE.._T._T."

In [0]:
data_endpoint = f"/data/{SELECTED_AGENCY_ID},{SELECTED_DFLOW_ID},{SELECTED_DFLOW_VER}/{QUERY_STR}"
data_params = {
    "format": "sdmx-json",
}

In [0]:
# get the response
data_response = requests.get(url=endpoint+data_endpoint, params=data_params).json()['data']

for list in data_response:
    print(list)

After retrieving the response in the JSON format, it's necessary to **connect the data points with their corresponding structure**. Those are separated in ```structure``` and ```dataSets```.

Let's first look into the ```structure```:

Using the ```dimensions``` key, we find two further keys: ```series``` which contains the **dimensions 1 to 5** and ```observation``` which contains **dimension number 6**, the ```TIME_PERIOD```. Those separated list indicate the different levels of attachment. 

In [0]:
# dimensions attached at series level
ser_dim_response = data_response['structure']['dimensions']['series']
pprint(ser_dim_response)

In [0]:
# dimensions attached at observation level
obs_dim_response = data_response['structure']['dimensions']['observation']
pprint(obs_dim_response)

Looking at those dimensions from the response, we can see the ```TIME_PERIOD``` which goes from 1950 to 2023, the ```REF_AREA``` which I limited to only return values for ```UNICEF_WE``` ('Western Europe'), all those different values for the ```INDICATOR```, and the ```RESIDENCE```, ```SEX``` and ```AGE``` all limited to 'Total'.

Next, let's get an overview of the attributes, which are also split into their different attachment levels: ```series``` and ```observation```. 

In [0]:
# attributes attached at series level
ser_attr_response = data_response['structure']['attributes']['series']
pprint(ser_attr_response)

In [0]:
# attributes attached at observation level
obs_attr_response = data_response['structure']['attributes']['observation']
pprint(obs_attr_response)

Now it's time to retrieve the actual data points:

In [0]:
dpoints_response = data_response['dataSets']
# to view uncomment the next line
# pprint(dpoints_response)

![data_points_detail.png](attachment:data_points_detail.png)

Each data series is uniquely identified by its **Key**, comprised of the dimensions attached at series level (in this instance: ```'0:0:0:0:0'```). In this case, only the second number changes because that is the number corresponding with the ```INDICATOR```.

In [0]:
# get the different keys of the data points and their corresponding dimensions  
for point in dpoints_response[0]['series']:
    print(f"{point} - "
          f"{ser_dim_response[0]['values'][int(point.split(':')[0])]['name']} : "
          f"{ser_dim_response[1]['values'][int(point.split(':')[1])]['name']} : "
          f"{ser_dim_response[2]['values'][int(point.split(':')[2])]['name']} : "
          f"{ser_dim_response[3]['values'][int(point.split(':')[3])]['name']} : "
          f"{ser_dim_response[4]['values'][int(point.split(':')[4])]['name']}")

Next are the attributes attached at series level. In this instance: ```[0, 0, 0, 0]```.

In [0]:
example = dpoints_response[0]['series']['0:0:0:0:0']
print(example)

In [0]:
for i, v in enumerate(example['attributes']):
    print(f"{ser_attr_response[i]['id']}: {v} -> {ser_attr_response[i]['values'][v]['name']}")

Finally, the concepts at the observation level: 

```'observations': {'0': ['360046.821', None, None, 0, None, None, None, None],}```

First, the dimensions attached at the observation level, so in this case, the ```TIME_PERIOD```.

The following list starts with the ```OBS_VALUE```, listed under ```primaryMeasure```. The other values correspond with the attributes at the observation-level.

In [0]:
print(f"'0' -> TIME_PERIOD: {obs_dim_response[0]['values'][0]['name']}")

In [0]:
obs_attr_response[0]

In [0]:
for i, v in enumerate(example['observations']['0']):
    if i == 0: 
        print(f"OBS_VALUE: {v}\n")
    else:
        print(f"{obs_attr_response[i-1]['name']}: {v}")

## **4. STEP: put the data into a Pandas DataFrame**

Let's explore how the number of births in Western Europe changed over the years. To do this I'll first create a DataFrame with all the useful values. 

In [0]:
# create a dictionary which will be converted into a pandas DataFrame later
data = {}

# going over all those dimension keys 
for k, v in dpoints_response[0]['series'].items():
    # because only the 'INDICATOR' changes I will only keep that 
    indicator_index = int(k.split(':')[1])

    # this will get the 'id' of the corresponding value in the 'INDICATOR' dimension 
    # this id will be used as column later and therefore used as a key in the new dict 
    indicator_key = ser_dim_response[1]['values'][indicator_index]['id']

    # to get the values I'll loop over the different keys for the observation dimension corresponding with the 'TIME_PERIOD'
    # I'll only use the observation values and put them into a consecutive list 
    # later the years will be used a indices 
    indicator_values = []
    for i, j in enumerate(obs_dim_response[0]['values']):
        try: 
            indicator_values.append(dpoints_response[0]['series'][k]['observations'][str(i)][0])
        except KeyError:
            indicator_values.append(None)
    
    # append the attributes to preserve additional information 
    for g, h in enumerate(v['attributes']):
        try:
            indicator_values.append(ser_attr_response[g]['values'][h]['name'])
        except TypeError:
            indicator_values.append(None)
    
    # put everything together 
    data[indicator_key] = indicator_values

print(data)


In [0]:
# create indices list for the DataFrame with years from 'TIME_PERIOD' dimension 
indices = [int(obs_dim_response[0]['values'][i]['id']) for i, v in enumerate(obs_dim_response[0]['values'])]

# create indices for the attributes that were added to the end 
attr_indices = [n['name'] for n in ser_attr_response]

# create the DataFrame
df = pd.DataFrame(data, index=indices+attr_indices)

df

In [0]:
df.info()

Because all those values were treated as if they were strings the Dtypes are objects. This has to be changed in order to work with them as numerical values. I will also drop the ```DM_POP_GRT``` and ```DM_POP_U_GRT``` because they only contain 2 NaN values which is not very useful for time comparison. 

In [0]:
new_df = df.copy()

# drop columns with only 2 NaN values 
new_df.drop(columns=['DM_POP_GRT', 'DM_POP_U_GRT'], axis=1, inplace=True)

While the last four rows are helpful to get more information, like the unit measure and unit multiplier, it has to be dropped to do analysis. 

In [0]:
new_df.drop(index=attr_indices, axis=0, inplace=True)

In [0]:
# change Dtypes 
for col in new_df.columns:
    new_df[col] = pd.to_numeric(new_df[col])

new_df.info()

Now I it's possible to explore how the number of births in Western Europe changed over the years. 

In [0]:
fig, ax = plt.subplots(figsize=(17,4))
plt.bar(
    x=new_df.index,
    height=new_df['DM_BRTS'],
    color='orange',
)

ax.grid(True, color="lightgrey")
plt.title('Number of Births from 1950 to 2023')
plt.ylabel('Number of Births in k')
plt.xlabel('Year')
plt.xlim(1950, 2023)

plt.show()

## **5. STEP: export desired data**

The DataFrame that was just created can easily be exported to CSV or Excel format for further data visualization with other tools. 

In [0]:
# export to csv
df.to_csv("my_sdmx_data.csv")

In [0]:
%pip install openpyxl
# export as Excel file 
df.to_excel("my_sdmx_data.xlsx")

### **Read the article for further explanation:** https://medium.com/@schwebel.felix/sdmx-rest-web-service-eafe4107117c

**Useful Links and Sources:**
- https://sdmx.data.unicef.org/overview.html
- https://sdmx.org/
- https://ec.europa.eu/eurostat/web/sdmx-infospace/trainings-tutorials
- https://data.unicef.org/sdmx-api-documentation/